# Simulation
### 1. Simulate trees with branch lengths 
```
Rscript alisim_input_generate.r -d exp,0.01 -l 1000 -t '((A,B),C,D);' -n 10000 -m topo1
Rscript alisim_input_generate.r -d exp,0.01 -l 1000 -t '((A,C),B,D);' -n 10000 -m topo2
Rscript alisim_input_generate.r -d exp,0.01 -l 1000 -t '((A,D),B,C);' -n 10000 -m topo3
``` 
### 2. Aggregate tree topologies 
```
cat experiment_exp_0.01_4taxa_topo*/TRAIN/train_simulation.tre > TRAIN_all.tre
cat experiment_exp_0.01_4taxa_topo*/TEST/test_simulation.tre > TEST_all.tre
Rscript partition_generate.r -n 30000 -l 1000 -f TRAIN
Rscript partition_generate.r -n 30000 -l 1000 -f TEST
```

### 3. Simulate alignments
```
./iqtree-2.3.2-macOS-intel/bin/iqtree2 --alisim TRAIN_all_concat -Q TRAIN.part -t TRAIN_all.tre -m JC --out-format fasta
./iqtree-2.3.2-macOS-intel/bin/iqtree2 --alisim TEST_all_concat -Q TEST.part -t TEST_all.tre -m JC --out-format fasta
```

### 4. Split result into multiple fasta files
```
awk '{print$2$3$5}' TRAIN.part | tail -n +3 > TRAIN.amas_part
python ./AMAS-master/amas/AMAS.py split -f fasta -i TRAIN_all_concat.fa -d dna -l TRAIN.amas_part -u fasta
find . -type f -name "TRAIN*out.fas" | sort -V | xargs cat > TRAIN_all.fasta
find . -type f -name "TRAIN*out.fas" | xargs rm

awk '{print$2$3$5}' TEST.part | tail -n +3 > TEST.amas_part
python ./AMAS-master/amas/AMAS.py split -f fasta -i TEST_all_concat.fa -d dna -l TEST.amas_part -u fasta
find . -type f -name "TEST*out.fas" | sort -V | xargs cat > TEST_all.fasta
find . -type f -name "TEST*out.fas" | xargs rm
```  

### 5. Convert fasta to numerical input
```
python fasta2numeric.py --tr TRAIN_all.fasta --te TEST_all.fasta 
```

### 6. Extract site patterns
```
python numeric2pattern.py --tr TRAIN.npy --te TEST.npy --co 4
```

# Machine learning models
### 1. Tree topology prediction using Random Forest (RF) algorithm
First, we are going to train RF and evaluate its performance on our TEST dataset. The RF model takes site patterns (X) and topology class (Y) as an input.     

In [ ]:
#Load numpy and machine learning module in python
from sklearn.ensemble import RandomForestClassifier
import numpy as np
#Define Random Forest Classifier 
clf = RandomForestClassifier(max_depth=2, random_state=0)
#Read numpy input TRAIN data X (i.e. our site patterns that we extracted from alignments)
X_train = np.load("TRAIN_sitepattern_4.npy")
#Let's check its shape
X_train.shape
#Create labels Y for the input data
Y_train = np.repeat(np.array([0,1,2]),10000)
#Train our classifier
clf.fit(X_train, Y_train)
#Read numpy input TEST data X
X_test = np.load("TEST_sitepattern_4.npy")
Y_test = Y_train
#Let's predict labels for the TEST data and get the accuracy
clf.score(X_test,Y_train)

### 2. Tree topology prediction using Multilayer perceptron (MLP)
Second, we will experiment with simple neural network arhitectures called Multilayer perceptrons or MLPs. The MLP takes site patterns (X) and topology class (Y) as an input.  

In [ ]:
import tensorflow as tf
import time
import sys, argparse, os
import numpy as np
from itertools import product, combinations
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, ZeroPadding2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

X_train = np.load("TRAIN_sitepattern_4.npy")
Y_train = to_categorical(np.repeat(np.array([0,1,2]),10000),num_classes=3)

# Length of feature vector, i.e. number of patterns 
N_patterns=X_train.shape[1]
    
#Number of tree topologies
N_labels = 3
    
visible_layer = Input(shape=(N_patterns,))
hidden1 = Dense(10,activation='relu')(visible_layer)
hidden2 = Dense(10,activation='relu')(hidden1)
output = Dense(N_labels, activation='softmax')(hidden2)
    

model_mlp = Model(inputs=visible_layer, outputs=output)
model_mlp.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
#Print model
print(model_mlp.summary())
   
#Model stopping criteria
callback=EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=1, mode='auto')
  
#Model training    
model_mlp.fit(x=X_train,y=Y_train,batch_size=50,callbacks=callback,epochs=50,verbose=1,shuffle=True,validation_split=0.1)

#Read numpy input TEST data X
X_test = np.load("TEST_sitepattern_4.npy")
Y_test = Y_train
#Let's predict labels for the TEST data and get the accuracy
evals_class = model_mlp.evaluate(X_test,Y_test,batch_size=100, verbose=1, steps=None)
print(evals_class[1])

### 3. Tree topology prediction using Convolutional Neural Nets (CNNs)
Finally, we perform topology estimation using Convolutional Neural Nets (CNNs). The CNN takes site patterns (X) and topology class (Y) as an input. 

In [ ]:
import tensorflow as tf
import time
import sys, argparse, os
import numpy as np
from math import factorial
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, ZeroPadding2D, Activation
from tensorflow.keras.layers import Conv2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

X_train = np.load("TRAIN.npy")
Y_train = to_categorical(np.repeat(np.array([0,1,2]),10000),num_classes=3)

# Length of MSA, i.e. number of sites 
Aln_length = X_train.shape[2]
# Number of taxa in MSA
Ntaxa = X_train.shape[1]
#Number of tree topologies
N_labels = 3
    
    
#1. MSA CNN branch 1 
#Hyperparameters
#Hight (horizontal)
conv_x=[Ntaxa,1,1,1,1,1,1,1]
#Width (vertical)
conv_y=[1,2,2,2,2,2,2,2]
pool=[1,4,4,4,2,2,2,1]
filter_s=[10,10,10,10,10,10]

conv_pool_n = 2
# CNN Arhitecture
visible_msa = Input(shape=X_train.shape[1:])
x = visible_msa
for l in list(range(0,conv_pool_n)):
    x = ZeroPadding2D(padding=((0, 0), (0,conv_y[l]-1)))(x)
    x = Conv2D(filters=filter_s[l], kernel_size=(conv_x[l], conv_y[l]), strides=1,activation='relu')(x)
    x = AveragePooling2D(pool_size=(1,pool[l]))(x)  

output_msa = Flatten()(x)
  
hidden1 = Dense(1000,activation='relu')(output_msa)
output = Dense(N_labels, activation='linear')(hidden1)
model_cnn = Model(inputs=visible_msa, outputs=output)
model_cnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
#Print model
print(model_cnn.summary())
   
#Model stopping criteria
callback=EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=1, mode='auto')
    
model_cnn.fit(x=X_train,y=Y_train,batch_size=200,callbacks=callback,epochs=30,verbose=1,shuffle=True,validation_split=0.1)

#Read numpy input TEST data X
X_test = np.load("TEST.npy")
Y_test = Y_train
#Let's predict labels for the TEST data and get the accuracy
evals_class = model_cnn.evaluate(X_test,Y_test,batch_size=100, verbose=1, steps=None)
print(evals_class[1])